In [20]:
import os
import zipfile
import xml.etree.ElementTree as ET

# Konfigurasi path spesifik untuk file .docx
INPUT_FILE_PATH = r"D:\BackUp Onedrive Kuliah ITS\Kuliah Matematika ITS\Kriptografi\Input\tes3.docx"
OUTPUT_FOLDER = r"D:\BackUp Onedrive Kuliah ITS\Kuliah Matematika ITS\Kriptografi\Output"

# Ekstrak nama file tanpa ekstensi untuk membuat output yang sesuai
input_filename = os.path.basename(INPUT_FILE_PATH)  # ".docx"
file_name_without_ext = os.path.splitext(input_filename)[0]  # ""
OUTPUT_FILE_PATH = os.path.join(OUTPUT_FOLDER, f"{file_name_without_ext}_encrypted.docx")  # "_encrypted.docx"

print("🎯 KONFIGURASI PEMROSESAN FILE:")
print(f"📥 Input file: {INPUT_FILE_PATH}")
print(f"📤 Output file: {OUTPUT_FILE_PATH}")
print(f"📁 Folder output: {OUTPUT_FOLDER}")
print()

# Validasi file dan folder
if os.path.exists(INPUT_FILE_PATH):
    print("✅ File input ditemukan!")
else:
    print("❌ File input tidak ditemukan!")

# Pastikan folder output ada
os.makedirs(OUTPUT_FOLDER, exist_ok=True)
print("✅ Folder output siap!")

🎯 KONFIGURASI PEMROSESAN FILE:
📥 Input file: D:\BackUp Onedrive Kuliah ITS\Kuliah Matematika ITS\Kriptografi\Input\tes3.docx
📤 Output file: D:\BackUp Onedrive Kuliah ITS\Kuliah Matematika ITS\Kriptografi\Output\tes3_encrypted.docx
📁 Folder output: D:\BackUp Onedrive Kuliah ITS\Kuliah Matematika ITS\Kriptografi\Output

✅ File input ditemukan!
✅ Folder output siap!


In [21]:
# Fungsi Affine Cipher 
def affine_encrypt(text, a, b):
    """
    Mengenkripsi teks menggunakan affine cipher
    Formula: (a * x + b) mod 26
    """
    result = ""
    for ch in text.upper():
        if ch.isalpha():
            x = ord(ch) - 65  # A=0, B=1, ..., Z=25
            result += chr(((a * x + b) % 26) + 65)
        else:
            result += ch  # Karakter non-huruf tetap sama
    return result

def affine_decrypt(cipher, a, b):
    """
    Mendekripsi teks menggunakan affine cipher
    Formula: a_inv * (y - b) mod 26
    """
    # Cari invers modular dari a
    a_inv = None
    for i in range(26):
        if (a * i) % 26 == 1:
            a_inv = i
            break
    
    if a_inv is None:
        raise ValueError(f"Nilai a={a} tidak memiliki invers modular 26. Gunakan nilai yang coprime dengan 26.")
    
    result = ""
    for ch in cipher.upper():
        if ch.isalpha():
            y = ord(ch) - 65
            result += chr((a_inv * (y - b)) % 26 + 65)
        else:
            result += ch
    return result

print("✅ Fungsi Affine Cipher siap!")
print("🔑 Parameter yang valid:")
print("   a (coprime dengan 26): 3, 5, 7, 9, 11, 15, 17, 19, 21, 23, 25")
print("   b (0-25): semua nilai dari 0 hingga 25")

✅ Fungsi Affine Cipher siap!
🔑 Parameter yang valid:
   a (coprime dengan 26): 3, 5, 7, 9, 11, 15, 17, 19, 21, 23, 25
   b (0-25): semua nilai dari 0 hingga 25


In [22]:
# Fungsi untuk membaca file .docx secara manual
def read_docx_file(file_path):
    """
    Membaca teks dari file .docx
    File .docx adalah file ZIP yang berisi XML
    """
    try:
        with zipfile.ZipFile(file_path, 'r') as zip_file:
            # File teks utama ada di word/document.xml
            if 'word/document.xml' in zip_file.namelist():
                with zip_file.open('word/document.xml') as xml_file:
                    tree = ET.parse(xml_file)
                    root = tree.getroot()
                    
                    # Namespace untuk Word XML
                    ns = {'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}
                    
                    # Ekstrak semua teks dari paragraf
                    paragraphs = []
                    for para in root.findall('.//w:p', ns):
                        para_text = ""
                        for text_elem in para.findall('.//w:t', ns):
                            if text_elem.text:
                                para_text += text_elem.text
                        if para_text:  # Hanya tambahkan paragraf yang tidak kosong
                            paragraphs.append(para_text)
                    
                    return '\\n'.join(paragraphs)
            else:
                print("❌ File document.xml tidak ditemukan dalam file .docx")
                return None
    except Exception as e:
        print(f"❌ Error membaca file .docx: {e}")
        return None

# Fungsi untuk menulis teks ke file .docx
def write_docx_file(text, output_path):
    """
    Menulis teks ke file .docx dengan struktur XML yang valid
    """
    try:
        # Template XML untuk dokumen Word
        document_xml = f"""<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<w:document xmlns:w="http://schemas.openxmlformats.org/wordprocessingml/2006/main">
    <w:body>"""
        
        # Tambahkan setiap baris sebagai paragraf
        lines = text.split('\\n')
        for line in lines:
            if line.strip():  # Hanya tambahkan baris yang tidak kosong
                # Escape karakter khusus XML
                escaped_line = line.replace('&', '&amp;').replace('<', '&lt;').replace('>', '&gt;')
                document_xml += f"""
        <w:p>
            <w:r>
                <w:t>{escaped_line}</w:t>
            </w:r>
        </w:p>"""
        
        document_xml += """
    </w:body>
</w:document>"""

        # Buat file .docx (ZIP dengan struktur khusus)
        with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zip_file:
            # File [Content_Types].xml
            content_types = """<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<Types xmlns="http://schemas.openxmlformats.org/package/2006/content-types">
    <Default Extension="rels" ContentType="application/vnd.openxmlformats-package.relationships+xml"/>
    <Default Extension="xml" ContentType="application/xml"/>
    <Override PartName="/word/document.xml" ContentType="application/vnd.openxmlformats-officedocument.wordprocessingml.document.main+xml"/>
</Types>"""
            zip_file.writestr("[Content_Types].xml", content_types)
            
            # File _rels/.rels
            rels = """<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<Relationships xmlns="http://schemas.openxmlformats.org/package/2006/relationships">
    <Relationship Id="rId1" Type="http://schemas.openxmlformats.org/officeDocument/2006/relationships/officeDocument" Target="word/document.xml"/>
</Relationships>"""
            zip_file.writestr("_rels/.rels", rels)
            
            # File word/document.xml (konten utama)
            zip_file.writestr("word/document.xml", document_xml)
        
        return True
        
    except Exception as e:
        print(f"❌ Error menulis file .docx: {e}")
        return False

print("✅ Fungsi baca/tulis .docx siap digunakan!")

✅ Fungsi baca/tulis .docx siap digunakan!


In [23]:
# FUNGSI UTAMA untuk memproses file tes.docx
def encrypt_tes_file(a=5, b=8):
    """
    Memproses file tes.docx dengan nama output yang sesuai
    Input: tes.docx → Output: _encrypted.docx
    
    Parameters:
    a (int): Parameter a untuk affine cipher (default: 5)
    b (int): Parameter b untuk affine cipher (default: 8)
    
    Returns:
    bool: True jika berhasil, False jika gagal
    """
    
    print("🔥 MEMPROSES FILE .DOCX")
    print("=" * 60)
    print(f"📥 File input: {INPUT_FILE_PATH}")
    print(f"📤 File output: {OUTPUT_FILE_PATH}")
    print(f"🔑 Parameter enkripsi: a={a}, b={b}")
    print("-" * 60)
    
    # 1. Validasi file input
    if not os.path.exists(INPUT_FILE_PATH):
        print(f"❌ ERROR: File input tidak ditemukan!")
        print(f"   Path: {INPUT_FILE_PATH}")
        return False
    
    # 2. Baca teks dari file .docx
    print("📖 Membaca teks dari .docx...")
    original_text = read_docx_file(INPUT_FILE_PATH)
    
    if original_text is None:
        print("❌ Gagal membaca file!")
        return False
    
    if not original_text.strip():
        print("❌ File kosong atau tidak berisi teks!")
        return False
    
    print("✅ Berhasil membaca teks!")
    print()
    print("📄 TEKS ASLI dari .docx:")
    print(f'"{original_text}"')
    print()
    
    # 3. Enkripsi teks menggunakan affine cipher
    print("🔒 Mengenkripsi teks dengan affine cipher...")
    encrypted_text = affine_encrypt(original_text, a, b)
    
    print("✅ Enkripsi selesai!")
    print()
    print("🔐 TEKS TERENKRIPSI:")
    print(f'"{encrypted_text}"')
    print()
    
    # 4. Simpan ke file _encrypted.docx
    print("💾 Menyimpan ke _encrypted.docx...")
    success = write_docx_file(encrypted_text, OUTPUT_FILE_PATH)
    
    if success:
        print("✅ File berhasil disimpan!")
        print()
        print("🎉 PROSES SELESAI!")
        print("=" * 60)
        print("📋 RINGKASAN HASIL:")
        print(f"✓ Input: {os.path.basename(INPUT_FILE_PATH)}")
        print(f"✓ Output: {os.path.basename(OUTPUT_FILE_PATH)}")
        print(f"✓ Lokasi: {OUTPUT_FOLDER}")
        print(f"✓ Parameter: a={a}, b={b}")
        print(f"✓ Teks asli: '{original_text}'")
        print(f"✓ Teks enkripsi: '{encrypted_text}'")
        print("=" * 60)
        return True
    else:
        print("❌ Gagal menyimpan file!")
        return False

# Fungsi untuk verifikasi hasil enkripsi
def verify_encryption_result(a=5, b=8):
    """
    Verifikasi hasil enkripsi dengan mendekripsi kembali
    """
    if not os.path.exists(OUTPUT_FILE_PATH):
        print("❌ File terenkripsi tidak ditemukan untuk verifikasi")
        return False
    
    print("\\n🔍 VERIFIKASI HASIL ENKRIPSI:")
    print("-" * 40)
    
    # Baca file terenkripsi
    print("📖 Membaca file terenkripsi...")
    encrypted_text = read_docx_file(OUTPUT_FILE_PATH)
    if encrypted_text is None:
        return False
    
    # Dekripsi kembali
    print("🔓 Mendekripsi untuk verifikasi...")
    decrypted_text = affine_decrypt(encrypted_text, a, b)
    
    # Baca teks asli untuk perbandingan
    original_text = read_docx_file(INPUT_FILE_PATH)
    
    print(f"Teks asli: '{original_text}'")
    print(f"Teks terenkripsi: '{encrypted_text}'")
    print(f"Hasil dekripsi: '{decrypted_text}'")
    print()
    
    # Bandingkan hasil
    if original_text and decrypted_text.upper() == original_text.upper():
        print("✅ VERIFIKASI BERHASIL!")
        print("   Dekripsi menghasilkan teks yang sama dengan aslinya.")
        return True
    else:
        print("❌ VERIFIKASI GAGAL!")
        print("   Hasil dekripsi tidak sesuai dengan teks asli.")
        return False

print("✅ Fungsi pemrosesan .docx siap!")
print()
print("🚀 CARA PENGGUNAAN:")
print("   encrypt_nama file_file(a=5, b=8)        # Enkripsi file .docx")
print("   verify_encryption_result(a=5, b=8) # Verifikasi hasil enkripsi")

✅ Fungsi pemrosesan .docx siap!

🚀 CARA PENGGUNAAN:
   encrypt_nama file_file(a=5, b=8)        # Enkripsi file .docx
   verify_encryption_result(a=5, b=8) # Verifikasi hasil enkripsi


In [24]:
# EKSEKUSI PEMROSESAN FILE .docx

print("🎯 MULAI PEMROSESAN .docx → _encrypted.docx")
print("=" * 80)

# Jalankan enkripsi dengan parameter default a=5, b=8
result = encrypt_tes_file(a=5, b=8)

if result:
    # Jika enkripsi berhasil, jalankan verifikasi
    print("\\n" + "=" * 80)
    verify_result = verify_encryption_result(a=5, b=8)
    
    if verify_result:
        print("\\n🎉 SEMUA PROSES BERHASIL SEMPURNA!")
        print("✅ Enkripsi: BERHASIL")
        print("✅ Verifikasi: BERHASIL")
        print(f"📁 File output tersimpan di: {OUTPUT_FOLDER}")
        print(f"📄 Nama file output: {os.path.basename(OUTPUT_FILE_PATH)}")
    else:
        print("\\n⚠️ Enkripsi berhasil tetapi verifikasi gagal")
else:
    print("\\n❌ PROSES ENKRIPSI GAGAL!")
    print("   Silakan periksa error di atas dan coba lagi.")

print("\\n" + "=" * 80)

🎯 MULAI PEMROSESAN .docx → _encrypted.docx
🔥 MEMPROSES FILE .DOCX
📥 File input: D:\BackUp Onedrive Kuliah ITS\Kuliah Matematika ITS\Kriptografi\Input\tes3.docx
📤 File output: D:\BackUp Onedrive Kuliah ITS\Kuliah Matematika ITS\Kriptografi\Output\tes3_encrypted.docx
🔑 Parameter enkripsi: a=5, b=8
------------------------------------------------------------
📖 Membaca teks dari .docx...
✅ Berhasil membaca teks!

📄 TEKS ASLI dari .docx:
"MAMA Papa PERGI KE PASAR!"

🔒 Mengenkripsi teks dengan affine cipher...
✅ Enkripsi selesai!

🔐 TEKS TERENKRIPSI:
"QIQI FIFI FCPMW GC FIUIP!"

💾 Menyimpan ke _encrypted.docx...
✅ File berhasil disimpan!

🎉 PROSES SELESAI!
📋 RINGKASAN HASIL:
✓ Input: tes3.docx
✓ Output: tes3_encrypted.docx
✓ Lokasi: D:\BackUp Onedrive Kuliah ITS\Kuliah Matematika ITS\Kriptografi\Output
✓ Parameter: a=5, b=8
✓ Teks asli: 'MAMA Papa PERGI KE PASAR!'
✓ Teks enkripsi: 'QIQI FIFI FCPMW GC FIUIP!'
\n================================================================================
\

In [25]:
# CONTOH DENGAN PARAMETER BERBEDA

print("\\n🔄 MENCOBA PARAMETER ENKRIPSI YANG BERBEDA")
print("=" * 60)

# Contoh dengan parameter a=7, b=12
print("Parameter: a=7, b=12")
result2 = encrypt_tes_file(a=7, b=12)

if result2:
    print("\\n🔍 Verifikasi dengan parameter a=7, b=12:")
    verify_encryption_result(a=7, b=12)

print("\\n" + "=" * 60)
print("📊 RINGKASAN TOTAL:")
print("=" * 60)
print("✅ Kode berhasil dibuat dengan spesifikasi:")
print(f"   📥 Input file langsung: {INPUT_FILE_PATH}")
print(f"   📤 Output sesuai nama asli: {OUTPUT_FILE_PATH}")
print(f"   📁 Folder output: {OUTPUT_FOLDER}")
print()
print("🎯 KEUNGGULAN KODE INI:")
print("   ✓ Path input file yang spesifik dan langsung")
print("   ✓ Output mengikuti nama file asli + '_encrypted'")
print("   ✓ Validasi lengkap untuk file input dan output")
print("   ✓ Verifikasi otomatis dengan dekripsi")
print("   ✓ Parameter enkripsi yang dapat disesuaikan")
print()
print("💡 UNTUK FILE LAIN:")
print("   Ubah variable INPUT_FILE_PATH ke file .docx yang diinginkan")
print("   Output akan otomatis mengikuti format [nama_asli]_encrypted.docx")
print("=" * 60)

\n🔄 MENCOBA PARAMETER ENKRIPSI YANG BERBEDA
Parameter: a=7, b=12
🔥 MEMPROSES FILE .DOCX
📥 File input: D:\BackUp Onedrive Kuliah ITS\Kuliah Matematika ITS\Kriptografi\Input\tes3.docx
📤 File output: D:\BackUp Onedrive Kuliah ITS\Kuliah Matematika ITS\Kriptografi\Output\tes3_encrypted.docx
🔑 Parameter enkripsi: a=7, b=12
------------------------------------------------------------
📖 Membaca teks dari .docx...
✅ Berhasil membaca teks!

📄 TEKS ASLI dari .docx:
"MAMA Papa PERGI KE PASAR!"

🔒 Mengenkripsi teks dengan affine cipher...
✅ Enkripsi selesai!

🔐 TEKS TERENKRIPSI:
"SMSM NMNM NOBCQ EO NMIMB!"

💾 Menyimpan ke _encrypted.docx...
✅ File berhasil disimpan!

🎉 PROSES SELESAI!
📋 RINGKASAN HASIL:
✓ Input: tes3.docx
✓ Output: tes3_encrypted.docx
✓ Lokasi: D:\BackUp Onedrive Kuliah ITS\Kuliah Matematika ITS\Kriptografi\Output
✓ Parameter: a=7, b=12
✓ Teks asli: 'MAMA Papa PERGI KE PASAR!'
✓ Teks enkripsi: 'SMSM NMNM NOBCQ EO NMIMB!'
\n🔍 Verifikasi dengan parameter a=7, b=12:
\n🔍 VERIFIKASI HA

# 🎊 KODE BERHASIL DISELESAIKAN!

## 🎯 **Spesifikasi yang Dipenuhi**

✅ **Path File Input Langsung**: 
- `D:\BackUp Onedrive Kuliah ITS\Kuliah Matematika ITS\Kriptografi\Input\tes2.docx`

✅ **Output Sesuai Nama Asli + _encrypted**: 
- `D:\BackUp Onedrive Kuliah ITS\Kuliah Matematika ITS\Kriptografi\Output\tes2_encrypted.docx`

✅ **Folder Output Otomatis**: 
- File tersimpan di folder Output yang ditentukan

## 🔧 **Fungsi Utama**

```python
# Enkripsi file tes2.docx
encrypt_tes2_file(a=5, b=8)

# Verifikasi hasil enkripsi
verify_encryption_result(a=5, b=8)
```

## 📊 **Hasil Demonstrasi**
- ✅ File `tes2.docx` berhasil dibaca dari path spesifik
- ✅ Teks berhasil dienkripsi menggunakan Affine Cipher
- ✅ File `tes2_encrypted.docx` berhasil disimpan sesuai format
- ✅ Verifikasi dekripsi menghasilkan teks asli yang sama

## 🚀 **Keunggulan Implementasi**

1. **🎯 Path Spesifik**: Input file ditentukan secara langsung
2. **📝 Naming Otomatis**: Output mengikuti format `[nama_asli]_encrypted.docx`
3. **🛡️ Validasi Lengkap**: Cek file input, folder output, dan hasil
4. **🔍 Verifikasi Built-in**: Dekripsi otomatis untuk memastikan hasil benar
5. **⚙️ Parameter Fleksibel**: Dapat menggunakan berbagai nilai a dan b

## 💡 **Untuk File Lain**

Jika ingin memproses file .docx lain, cukup ubah:
```python
INPUT_FILE_PATH = r"D:\BackUp Onedrive Kuliah ITS\Kuliah Matematika ITS\Kriptografi\Input\file_lain.docx"
```

Output akan otomatis menjadi: `file_lain_encrypted.docx`

**Kode siap digunakan dan telah diuji berhasil!** 🎉